# Step 01: Export to RDM

This notebook exports all analysis results and groups to a Results Data Mart (RDM).

**How it works:**
- Creates one job per analysis/group for individual export
- First job (seed job) creates the RDM and is polled to completion
- Remaining jobs append to the RDM in parallel

**Tasks:**
- Retrieve Export to RDM batch from Stage_01/Step_03
- Review export job configuration
- Submit seed job (creates RDM) and poll to completion
- Submit remaining jobs (append to RDM)

**Prerequisites:**
- All Analysis jobs must be complete
- All Grouping and Grouping Rollup jobs must be complete
- Export RDM Name must be configured in Metadata

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch, validate_batch
from helpers.database import execute_query
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_01_Export_to_RDM.ipynb')

# Display context
ux.header("Export to RDM")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Export to RDM Batch

In [ ]:
# Retrieve Export to RDM batch from Stage_01/Step_03
ux.subheader("Retrieve Export to RDM Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.EXPORT_TO_RDM not in batches:
    raise ValueError(f"Export to RDM batch not found. Available: {list(batches.keys())}")

export_batch_id = int(batches[BatchType.EXPORT_TO_RDM])

ux.success(f"Retrieved Export to RDM batch: ID={export_batch_id}")
step.log(f"Retrieved Export to RDM batch: ID={export_batch_id}")

## 3) Review Export Configuration

In [ ]:
# Read batch details and extract configuration
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(export_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch (one job per analysis/group)
jobs = get_batch_jobs(export_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Extract configuration from jobs - count analyses vs groups
rdm_name = 'N/A'
server_name = 'N/A'
analysis_job_count = 0
group_job_count = 0
all_item_names = []

for job in jobs:
    config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
    config_result = execute_query(config_query, (job['job_configuration_id'],))
    config_data = config_result.iloc[0]['job_configuration_data'] if not config_result.empty else {}
    
    # Get common info from first job
    if rdm_name == 'N/A':
        rdm_name = config_data.get('rdm_name', 'N/A')
        server_name = config_data.get('server_name', 'N/A')
    
    # Count by type - each job has is_group flag
    if config_data.get('is_group', False):
        group_job_count += 1
    else:
        analysis_job_count += 1
    
    # Collect item name (each job has one item in analysis_names)
    item_names = config_data.get('analysis_names', [])
    if item_names:
        all_item_names.append(item_names[0])

# Show export configuration details
ux.subheader("Export Configuration")
export_info = [
    ["Target RDM", rdm_name],
    ["Server", server_name],
    ["Analysis jobs", analysis_job_count],
    ["Group jobs", group_job_count],
    ["Total jobs", job_count]
]
ux.table(export_info, headers=["Property", "Value"])

ux.info("\nNote: Each job exports a single analysis or group.")
ux.info("First job creates the RDM, subsequent jobs append.")

# Show preview of items to export
if all_item_names:
    ux.info("\nItems to export (first 15):")
    for name in all_item_names[:15]:
        ux.info(f"  - {name}")
    if len(all_item_names) > 15:
        ux.info(f"  ... and {len(all_item_names) - 15} more")

## 4) Check RDM Existence and Handle Conflicts

In [ ]:
# Check if RDM already exists
ux.subheader("Check RDM Existence")

irp_client = IRPClient()

try:
    # Check if RDM exists
    rdm_full_name = irp_client.rdm.get_rdm_database_full_name(rdm_name, server_name)
    rdm_exists = True
    ux.warning(f"⚠ RDM '{rdm_name}' already exists")
    ux.info(f"   Full name: {rdm_full_name}")
    step.log(f"RDM '{rdm_name}' already exists (full name: {rdm_full_name})")
except Exception as e:
    rdm_exists = False
    ux.success(f"✓ RDM '{rdm_name}' does not exist - ready to create")
    step.log(f"RDM '{rdm_name}' does not exist - will create new RDM")

# If RDM exists, prompt user for deletion
should_delete = False
if rdm_exists:
    ux.info("")
    ux.info("The RDM you are trying to create already exists on the server.")
    ux.info("Would you like to delete the existing RDM and recreate it?")
    ux.info("")
    ux.warning("⚠ WARNING: Deleting the RDM will permanently remove all exported data.")
    ux.info("")

    response = input("Delete existing RDM and recreate? (y/n): ").strip().lower()

    if response == 'y':
        should_delete = True
        ux.info("\n✓ User confirmed RDM deletion")
        step.log("User confirmed deletion of existing RDM")
    else:
        should_delete = False
        ux.info("\n✓ User declined RDM deletion - step will complete without submission")
        step.log("User declined deletion of existing RDM")

## 5) Delete Existing RDM (if requested)

In [ ]:
# Delete existing RDM if user confirmed
delete_job_id = None

if rdm_exists and should_delete:
    ux.subheader("Delete Existing RDM")

    ux.info(f"Submitting delete job for RDM: {rdm_name}")

    try:
        # Submit delete job
        delete_job_id = irp_client.rdm.submit_delete_rdm_job(rdm_name, server_name)
        ux.success(f"✓ Delete job submitted successfully")
        ux.info(f"   Job ID: {delete_job_id}")
        step.log(f"Submitted delete RDM job: {delete_job_id}")

        # Poll job to completion
        ux.info("\nPolling delete job to completion...")
        ux.info("(This may take several minutes)")

        final_status = irp_client.rdm.poll_delete_rdm_job_to_completion(
            delete_job_id,
            interval=10,
            timeout=600000
        )

        ux.success(f"\n✓ Delete job completed successfully")
        ux.info(f"   Final status: {final_status}")
        step.log(f"Delete RDM job completed: {final_status}")

        ux.info(f"\nRDM '{rdm_name}' has been deleted - proceeding with export")

    except Exception as e:
        ux.error(f"\n✗ Failed to delete RDM: {e}")
        step.log(f"Failed to delete RDM: {e}")

        # Mark step as failed
        from helpers.step import update_step_run
        from helpers.constants import StepStatus
        update_step_run(step.run_id, StepStatus.FAILED, error_message=f"RDM deletion failed: {e}")

        raise RuntimeError(f"RDM deletion failed: {e}")

elif rdm_exists and not should_delete:
    ux.info("\nSkipping RDM deletion per user request")
else:
    ux.info("\nNo RDM deletion required - RDM does not exist")

## 6) Validate Batch and Review Jobs

In [ ]:
# Validate batch and categorize jobs (only if proceeding with submission)
validation_warnings = []

if rdm_exists and not should_delete:
    # User declined deletion - skip validation
    ux.warning("⏭ Skipping validation - user declined RDM deletion")
else:
    # Check entity existence for FINISHED/CANCELLED jobs to show what will be resubmitted
    from helpers.entity_validator import EntityValidator
    from helpers.constants import JobStatus

    validator = EntityValidator()

    # Categorize jobs by status and entity existence
    pending_jobs = []      # INITIATED jobs - will be submitted
    skipped_jobs = []      # FINISHED/CANCELLED jobs where entity still exists - will be skipped
    resubmit_jobs = []     # FINISHED/CANCELLED jobs where entity is missing - will be resubmitted
    other_jobs = []        # Jobs in other states (SUBMITTED, RUNNING, etc.)

    for job in jobs:
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        config_data = config_result.iloc[0]['job_configuration_data'] if not config_result.empty else {}
        
        # Get item name from job config (each job has one item)
        item_names = config_data.get('analysis_names', [])
        item_name = item_names[0] if item_names else 'Unknown'
        display_name = f"Export '{item_name}' to {rdm_name}"
        
        if job['status'] == JobStatus.INITIATED:
            pending_jobs.append(display_name)
        elif job['status'] in (JobStatus.FINISHED, JobStatus.CANCELLED):
            entity_exists = validator.check_entity_exists_for_job(config_data, batch['batch_type'])
            if entity_exists:
                skipped_jobs.append(display_name)
            else:
                resubmit_jobs.append(display_name)
        else:
            other_jobs.append((display_name, job['status']))

    # Display categorized job summary
    ux.subheader("Job Summary")

    if pending_jobs:
        ux.info(f"\n🆕 Jobs to submit ({len(pending_jobs)}):")
        for name in pending_jobs[:10]:
            ux.info(f"  • {name}")
        if len(pending_jobs) > 10:
            ux.info(f"  ... and {len(pending_jobs) - 10} more")

    if resubmit_jobs:
        ux.warning(f"\n🔄 Jobs to resubmit - RDM data missing ({len(resubmit_jobs)}):")
        for name in resubmit_jobs[:10]:
            ux.warning(f"  • {name}")
        if len(resubmit_jobs) > 10:
            ux.warning(f"  ... and {len(resubmit_jobs) - 10} more")

    if skipped_jobs:
        ux.success(f"\n✓ Jobs to skip - RDM data exists ({len(skipped_jobs)}):")
        for name in skipped_jobs[:10]:
            ux.success(f"  • {name}")
        if len(skipped_jobs) > 10:
            ux.success(f"  ... and {len(skipped_jobs) - 10} more")

    if other_jobs:
        ux.info(f"\n⏳ Jobs in progress ({len(other_jobs)}):")
        for name, status in other_jobs[:5]:
            ux.info(f"  • {name} ({status})")

    # Summary line
    total_to_process = len(pending_jobs) + len(resubmit_jobs)
    ux.info(f"\n📊 Summary: {total_to_process} job(s) will be submitted, {len(skipped_jobs)} will be skipped")

    # Validate batch before submission
    ux.subheader("Validate Batch")
    validation_messages = validate_batch(export_batch_id)

    # For Export to RDM with single-item jobs, treat ALL validation messages as warnings
    # Missing analyses/groups will be skipped during export, not treated as blocking errors
    validation_warnings = validation_messages  # All messages are warnings

    # Display warnings (missing items that will be skipped)
    if validation_warnings:
        ux.warning(f"\n⚠ Validation warnings ({len(validation_warnings)}):")
        for warning in validation_warnings[:15]:
            ux.warning(f"  {warning}")
        if len(validation_warnings) > 15:
            ux.warning(f"  ... and {len(validation_warnings) - 15} more")
        ux.info("\n  Missing items will be skipped during export.")
        step.log(f"Validated batch with {len(validation_warnings)} warning(s): {total_to_process} job(s) to submit, {len(skipped_jobs)} to skip")
    else:
        ux.success("\n✓ Batch validation passed - all items found")
        step.log(f"Verified batch: {total_to_process} job(s) to submit, {len(skipped_jobs)} to skip")

## 7) Submit Export to RDM Batch

This step submits the batch using a two-phase approach:
1. **Seed job**: The first job creates the RDM and is polled to completion
2. **Remaining jobs**: Once the RDM exists, all other jobs append in parallel

In [ ]:
# Submit batch to Moody's API
if rdm_exists and not should_delete:
    ux.warning("⏭ Skipping submission - user declined RDM deletion")
    submit_result = None
    failed_count = 0
    user_declined_deletion = True
else:
    ux.subheader("Submit Batch to Moody's")

    ux.info("")
    ux.info("Submission Process:")
    ux.info("  - Analysis and group names will be resolved to URIs")
    ux.info("  - Results will be exported to the configured RDM")
    ux.info(f"  - Target RDM: {rdm_name}")
    ux.info(f"  - Server: {server_name}")
    ux.info("  - Jobs will transition to SUBMITTED status")
    ux.info("  - Batch will transition to ACTIVE status")
    if validation_warnings:
        ux.info(f"  - {len(validation_warnings)} job(s) with missing items will be skipped")
    ux.info("")

    # Submit
    ux.info("\nSubmitting batch...")

    # Pass step.step_id to associate batch with this step (not the creation step)
    submit_result = submit_batch(export_batch_id, IRPClient(), step_id=step.step_id)

    # Display results
    ux.success(f"\nBatch submission completed")
    ux.info(f"  Submitted: {submit_result['submitted_jobs']} job(s)")
    ux.info(f"  Status: {submit_result['batch_status']}")

    # Check for submission errors (API failures, not missing analyses)
    failed_count = len([j for j in submit_result['jobs'] if 'error' in j])
    if failed_count > 0:
        ux.warning(f"\n{failed_count} job(s) had submission errors")
        for job_result in submit_result['jobs']:
            if 'error' in job_result:
                ux.warning(f"  Job {job_result['job_id']}: {job_result['error']}")

    step.log(f"Batch submitted: {submit_result['submitted_jobs']} job(s), {failed_count} with errors")
    user_declined_deletion = False

## 8) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

if user_declined_deletion:
    # Handle user declining RDM deletion - complete step successfully
    output_data = {
        'batch_id': export_batch_id,
        'batch_type': batch['batch_type'],
        'rdm_name': rdm_name,
        'server_name': server_name,
        'rdm_already_exists': True,
        'user_declined_deletion': True,
        'submitted': False
    }
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("STEP COMPLETED - NO SUBMISSION")
    ux.success("="*60)
    ux.info(f"\nRDM '{rdm_name}' already exists on server")
    ux.info("User declined deletion - no submission performed")
    ux.info("\nThe existing RDM has been preserved.")
    ux.info("If you need to recreate the RDM, re-run this notebook and choose 'y' when prompted.")

else:
    # Complete the step successfully (even with warnings or skipped jobs)
    # Collect all warnings for documentation
    all_warnings = validation_warnings.copy() if validation_warnings else []
    
    # Add submission errors as warnings (jobs that failed to submit)
    if failed_count > 0:
        failed_job_warnings = [
            f"Job {j['job_id']} submission error: {j['error']}" 
            for j in submit_result['jobs'] if 'error' in j
        ]
        all_warnings.extend(failed_job_warnings)
    
    output_data = {
        'batch_id': export_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': submit_result['batch_status'],
        'submitted_jobs': submit_result['submitted_jobs'],
        'skipped_jobs': failed_count,
        'rdm_name': rdm_name,
        'server_name': server_name,
        'total_items_configured': len(all_item_names),
        'analysis_job_count': analysis_job_count,
        'group_job_count': group_job_count,
        'rdm_deleted': delete_job_id is not None,
        'delete_job_id': delete_job_id,
        'warnings': all_warnings if all_warnings else None
    }
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("EXPORT TO RDM SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    
    if delete_job_id:
        ux.info(f"\nExisting RDM was deleted (Job ID: {delete_job_id})")
    
    ux.info(f"\nSubmitted {submit_result['submitted_jobs']} job(s) to Moody's API")
    ux.info(f"Target RDM: {rdm_name}")
    ux.info(f"Items configured: {len(all_item_names)} ({analysis_job_count} analyses + {group_job_count} groups)")
    ux.info(f"Batch status: {submit_result['batch_status']}")
    
    if all_warnings:
        ux.warning(f"\n⚠ Completed with {len(all_warnings)} warning(s):")
        for warning in all_warnings[:10]:
            ux.warning(f"  • {warning}")
        if len(all_warnings) > 10:
            ux.warning(f"  ... and {len(all_warnings) - 10} more")
        ux.info("\nMissing items were skipped during export.")
    
    ux.info("\nMonitor job progress and verify export completion in Moody's.")